In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../../../')))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
height = 128
width = 128
size = height*width
windows_size = 5
path = "C:\VScode_Projects\DP\datasets\CIC-IDS-2017\clean"
image_dir_name = "\image_inc\\"
csv_file = path + "\cicids2017_img_inc.csv"

In [ ]:
df = pd.read_csv(path+"\CICIDS_converted_data_final_2.csv")

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.protocol.unique()

In [ ]:
df.protocol.value_counts()

In [ ]:
df.label.unique()

In [ ]:
len(df.label.unique())

In [ ]:
df.label.value_counts() / len(df)

In [ ]:
df.label.value_counts()

In [ ]:
df = df.to_numpy()
df.shape

In [ ]:
# normalize payload, ttl, total_len
for col_index in range(df.shape[1]-7):
    column = df[:, col_index]
    column_normalized = (column - np.min(column)) / (np.max(column) - np.min(column))
    df[:, col_index] = column_normalized

In [ ]:
# inspect t_delta
print(np.min(df[:, -6]), np.max(df[:, -6]))

In [ ]:
# normalize t_delta
column = df[:, -6]
column_normalized = (column - np.min(column)) / (np.max(column) - np.min(column))
df[:, -6] = column_normalized

In [ ]:
print(np.min(df[:, -6]), np.max(df[:, -6]))

In [ ]:
#inspect label
np.unique(df[:, -5])

In [ ]:
def convert_dataset_to_grayscale_image(arr):
    for idx, row in enumerate(arr):
        print(row.shape)
        data = row[:-4]
        print(data.shape)
        idset = list(row[-4:]) + [row[-7]]
        srcip = idset[0]
        idset = frozenset(idset)
        print(idset, srcip)
        data = data[:-1]
        print(data.shape)
        label = data[-1]
        print(label)
        
        # data = np.pad(data, pad_width=int((size-len(data))/2), constant_values=0)
        # print(data.shape)
        # data = data.reshape(height, width).astype('float64')
        # print(data.shape)
        
        # plt.imshow(data, cmap='gray')
        # plt.axis('off')  # Remove axes
        # plt.show()
        
        break
    print("DONE")
        
convert_dataset_to_grayscale_image(df)

In [ ]:
%run ../../../../sniffer/flow.py

In [ ]:
from sniffer.flow import FlowControl

In [ ]:
def convert_dataset_to_rgb_image(arr):
    flows = FlowControl()
    for idx, row in enumerate(arr):
        print(row.shape)
        idset = list(row[-4:]) + [row[-7]]
        srcip = idset[0]
        idset = frozenset(idset)
        print(idset, srcip)
        data = row[:-5]
        label = row[-5]
        print(data.shape)
        print(label)
        
        data, _ = flows.attach_dict(idset=idset, data=data, srcip=srcip, window_size=windows_size)
        data = np.array(data).flatten()
        
        data = np.pad(data, pad_width=int((size-len(data))/2), constant_values=0)
        data = data.reshape(height, width)
        print(data.shape)
        
        channel_1 = data.astype('float64')
        print(channel_1.shape)
        channel_2 = np.rot90(channel_1, k=2).reshape(height, width)
        print(channel_2.shape)
        channel_3 = np.rot90(channel_2, k=2).reshape(height, width)
        print(channel_3.shape)
        img = np.stack((channel_1, channel_2, channel_3)).transpose((1, 2, 0))
        print(img.shape)
        
        plt.imshow(img)
        plt.axis('off')  # Remove axes
        plt.show() 
        break
    
    print("DONE")
        
convert_dataset_to_rgb_image(df)

In [ ]:
import threading
import queue

image_queue = queue.Queue()
def save_images():
    while True:
        filename, image = image_queue.get()
        if image is None:
            print("THREAD DEATH")
            break
        # Save the image
        cv2.imwrite(filename, image)
        image_queue.task_done()

with open(csv_file, 'w') as f:
    np.savetxt(f, [np.array(["file_name", "label"])], delimiter=',', fmt='%s')

def convert_dataset_to_image(arr):
    flows = FlowControl()
    image_saver = threading.Thread(target=save_images, daemon=True)
    with open(csv_file, 'a') as f:
        for idx, row in enumerate(arr):
            idset = list(row[-4:]) + [row[-7]]
            srcip = idset[0]
            idset = frozenset(idset)
            data = row[:-5]
            label = row[-5]
                    
            data, _ = flows.attach_dict(idset=idset, data=data, srcip=srcip, window_size=windows_size)
            
            data = np.array(data).flatten()
            data = np.pad(data, pad_width=int((size-len(data))/2), constant_values=0)
            data = data.reshape(height, width)
            
            channel_1 = data.astype('float64')
            channel_2 = np.rot90(channel_1, k=2).reshape(height, width)
            channel_3 = np.rot90(channel_2, k=2).reshape(height, width)
            
            img = np.stack((channel_1, channel_2, channel_3)).transpose((1, 2, 0))
            file_name = f"cic_ids_2017_{idx}.png"
            # cv2.imwrite(path+image_dir_name+file_name, img*255)
            image_queue.put((path+image_dir_name+file_name, img*255))
        
            log = np.array([file_name, label])
            np.savetxt(f, [log], delimiter=',', fmt='%s')
            
            if idx % 100_000 == 0:
                print(idx, label)
                if not image_saver.is_alive():
                    print("THREAD RESTART")
                    image_saver.start
            
    print("DONE")
        

convert_dataset_to_image(df)